In [1]:
# Part 1 - Building the CNN

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.models import model_from_json

/home/abhinandan/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Initialising the CNN
classifier = Sequential()

In [3]:
# Step 1 - Convolution
classifier.add(Conv2D(32, (8, 8), input_shape = (64, 64, 3), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

In [4]:
# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

In [5]:
# Step 3 - Flattening
classifier.add(Flatten())

In [6]:
# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

In [7]:
# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [8]:
# Part 2 - Fitting the CNN to the images

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [9]:
classifier.fit_generator(training_set,
                         steps_per_epoch =100,
                         epochs = 25,
                         validation_data = test_set,
                         validation_steps = 10)

Epoch 1/25
100/100 [==============================] - 90s 897ms/step - loss: 0.6956 - acc: 0.5228 - val_loss: 0.6942 - val_acc: 0.4813
Epoch 2/25
100/100 [==============================] - 58s 581ms/step - loss: 0.6931 - acc: 0.5091 - val_loss: 0.6925 - val_acc: 0.5188
Epoch 3/25
100/100 [==============================] - 46s 461ms/step - loss: 0.6939 - acc: 0.5009 - val_loss: 0.6911 - val_acc: 0.5312
Epoch 4/25
100/100 [==============================] - 25s 253ms/step - loss: 0.6933 - acc: 0.4888 - val_loss: 0.6929 - val_acc: 0.5531
Epoch 5/25
100/100 [==============================] - 26s 261ms/step - loss: 0.6923 - acc: 0.5266 - val_loss: 0.6926 - val_acc: 0.5406
Epoch 6/25
100/100 [==============================] - 26s 264ms/step - loss: 0.6932 - acc: 0.4950 - val_loss: 0.6920 - val_acc: 0.4813
Epoch 7/25
100/100 [==============================] - 27s 266ms/step - loss: 0.6898 - acc: 0.5291 - val_loss: 0.6852 - val_acc: 0.5344
Epoch 8/25
100/100 [==============================] - 2

In [10]:
training_set.class_indices

{'cats': 0, 'dogs': 1}

In [20]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_2.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)

if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'

In [21]:
prediction

'dog'

In [12]:
# Saving a Keras Model
classifier_json = classifier.to_json()
with open("classifier.json", "w") as json_file:
    json_file.write(classifier_json)
# Serialize weights to HDF5
print("Saved model to disk")
classifier.save_weights("classifier.h5")

Saved model to disk


In [13]:
# Load json and create model
json_file = open('classifier.json', 'r')
loaded_classifier_json = json_file.read()
json_file.close()
loaded_classifier = model_from_json(loaded_classifier_json)
# load weights into new model
print("Loaded model from disk")
loaded_classifier.load_weights("classifier.h5")

Loaded model from disk


In [14]:
classifier = loaded_classifier

## Transfer Learning

In [2]:
from tensorflow.python.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D

num_classes = 2

In [2]:
my_new_model = Sequential()
my_new_model.add(ResNet50(include_top=False, pooling='max'))
my_new_model.add(Dense(num_classes, activation='softmax'))

94658560/94653016 [==============================] - 541s 6us/step


In [3]:
my_new_model.layers[0].trainable = False

In [4]:
# compile
my_new_model.compile(optimizer='adam', 
                     loss='categorical_crossentropy', 
                     metrics=['accuracy'])

In [ ]:
my_new_model.fit_generator(training_set,
                         steps_per_epoch =100,
                         epochs = 25,
                         validation_data = test_set,
                         validation_steps = 10)